[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/sparse/splade/splade-embedding-generation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/sparse/splade/splade-embedding-generation.ipynb)

# SPLADE Sparse-Dense Embedding Generation

## Overview

SPLADE is a class of models that produce sparse embeddings. Unlike dense embeddings which can be difficult to interpret sparse embeddings map to tokens for easier interpretability. SPLADE models have been shown to consistently outperform dense models, particularly in out-of-domain settings.

The following guide will show you how to construct SPLADE embeddings to use with Pinecone's sparse-dense index. See the [companion guide](https://github.com/pinecone-io/examples/blob/master/sparse/splade/splade-queries.ipynb) to learn to skip embedding generation

## Install & Import

In [ ]:
!pip install -qU \
          git+https://git@github.com/pinecone-io/pinecone-python-client.git#egg=pinecone-client[grpc] \
          transformers \
          torch \
          sentence_transformers

In [ ]:
import pandas as pd

## Sparse Embeddings with SPLADE

In the following example we will use. SPLADE Model: [naver/splade-cocondenser-ensembledistil](https://huggingface.co/naver/splade-cocondenser-ensembledistilhttps://huggingface.co/naver/splade-cocondenser-ensembledistil)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

class SPLADE:
    def __init__(self, model):
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.model = AutoModelForMaskedLM.from_pretrained(model)

    def __call__(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt")

        with torch.no_grad():
            logits = self.model(**inputs).logits

        inter = torch.log1p(torch.relu(logits[0]))
        token_max = torch.max(inter, dim=0)  # sum over input tokens
        nz_tokens = torch.where(token_max.values > 0)[0]
        nz_weights = token_max.values[nz_tokens]

        order = torch.sort(nz_weights, descending=True)
        nz_weights = nz_weights[order[1]]
        nz_tokens = nz_tokens[order[1]]
        return {'indices': nz_tokens.numpy().tolist(), 'values': nz_weights.numpy().tolist()}

In [ ]:
splade = SPLADE("naver/splade-cocondenser-ensembledistil")

In [ ]:
doc = "what is the capital of france?"
sparse_vector = splade(doc)

SPLADE query & document expansion

In [ ]:
pd.DataFrame({
    **sparse_vector,
    **{'tokens': splade.tokenizer.convert_ids_to_tokens(sparse_vector['indices'])} # Fetch original tokens
})[:10]

## Dense Embeddings

For dense embeddings we use a popular model from sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)

## Processing the Quora Dataset

Quora is a popular dataset for evaluating text search

In [ ]:
import pandas as pd

sample = 1000
df = pd.read_parquet('https://storage.googleapis.com/gareth-pinecone-datasets/quora_all-MiniLM-L6-splade_v2.parquet', columns=['id', 'text'])
df = df.sample(sample)

In [ ]:
df.head(5)

### Create Embeddings

In [ ]:
%%time
df['sparse_values'] = df['text'].apply(lambda x: splade(x))

In [ ]:
%%time
df['values'] = df['text'].apply(lambda x: model.encode(x).tolist())

## Upsert to Pinecone

In [ ]:
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(api_key="YOUR_API_KEY", environment="YOUR_ENVIRONMENT")

In [ ]:
index_name = "splade-embedding-generation"
batch_size = 300
dimension = 384

In [ ]:
pinecone.create_index(
    index_name,
    pod_type='s1',
    metric='dotproduct',
    dimension=dimension,
    metadata_config={"indexed": []}
)

In [ ]:
from tqdm import tqdm
from pinecone import GRPCVector, GRPCSparseValues
from google.protobuf.struct_pb2 import Struct

with pinecone.GRPCIndex(index_name) as index:
    for i in tqdm(range(0, len(df), batch_size)):
        batch = df[i:min(i+batch_size, len(df))].to_dict(orient='records')
        upserts = []
        for row in batch:
            metadata = Struct()
            metadata.update(dict(text=row['text']))
            u = GRPCVector(
                id=str(row['id']),
                values=row['values'],
                metadata=metadata,
                sparse_values=GRPCSparseValues(
                    indices=row['sparse_values']['indices'],
                    values=row['sparse_values']['values']
                )
            )
            upserts.append(u)
        index.upsert(vectors=upserts, async_req=False)

## Querying

In [ ]:
index = pinecone.Index(index_name)

In [ ]:
index.query(id="1", top_k=10)

## Hybrid Queries with SPLADE

We can fetch records by calculating distance over both sparse and dense vectors

In [ ]:
def hybrid_score_norm(dense, sparse, alpha: float):
    """Hybrid score using a convex combination
    """
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    hs = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    return [v * alpha for v in dense], hs

In [ ]:
from pinecone import SparseValues

index = pinecone.Index(index_name)

In [ ]:
text = "nyc bites"
sparse = splade(text)
dense = model.encode(text).tolist()

### Sparse Only

In [ ]:
hdense, hsparse = hybrid_score_norm(dense, sparse, 0.0)
index.query(top_k=3, vector=hdense, sparse_vector=SparseValues(**hsparse), include_metadata=True)['matches']

### Hybrid

In [ ]:
hdense, hsparse = hybrid_score_norm(dense, sparse, 0.25)
index.query(top_k=3, vector=hdense, sparse_vector=SparseValues(**hsparse), include_metadata=True)['matches']

### Only Dense

In [ ]:
hdense, hsparse = hybrid_score_norm(dense, sparse, 1.0)
index.query(top_k=3, vector=hdense, sparse_vector=SparseValues(**hsparse), include_metadata=True)['matches']